<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/test/notebooks/NeuralBeagle_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install -qU transformers accelerate

In [12]:
from transformers import AutoTokenizer
import transformers
import torch

In [13]:
function = """
def checkGPU(tensorflow):
    if tensorflow == True:
        import tensorflow as tf
        print("Number of GPUs available with tensorflow:", len(tf.config.list_physical_devices('GPU')))
    else:
        import torch
        print('Checking if the GPU is available with PyTorch:', torch.cuda.is_available())
"""

In [14]:
model = "mlabonne/NeuralBeagle14-7B"

In [15]:
messages = [
    {
        "role": "system",
        "content": "You are an informative programming assistant that understands the working of a function and replies with it's documentation in markdown syntax."
    },
    {
        "role": "user",
        "content": f"{function}"
    }
]

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model)
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16, # Remember that float32 - CPU and float16 - GPU, keep this in mind if RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'
    device_map="auto",
)

tokenizer_config.json: 100%|██████████| 1.25k/1.25k [00:00<00:00, 389kB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 3.60MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:02<00:00, 771kB/s]
config.json: 100%|██████████| 637/637 [00:00<00:00, 238kB/s]
model.safetensors.index.json: 100%|██████████| 23.9k/23.9k [00:00<00:00, 1.87MB/s]
model-00001-of-00003.safetensors:  34%|███▍      | 1.68G/4.94G [07:44<15:04, 3.61MB/s]


KeyboardInterrupt: 

In [ ]:
%%time
outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>user
What is a large language model?</s>
<s>assistant
A large language model is an artificial intelligence system designed to process and generate human language, typically in the form of text. These models are based on deep learning architectures, particularly neural networks, and are trained on vast amounts of text data from various sources, such as books, websites, and social media. Large language models are capable of understanding and generating text that closely resembles human writing, with the ability to comprehend context, grammar, and meaning. Some well-known examples of large language models include GPT-3 (Generative Pre-trained Transformer 3) by OpenAI, BERT (Bidirectional Encoder Representations from Transformers) by Google, and T5 (Text-to-Text Transfer Transformer) by Google AI Language. These models have found applications in natural language processing tasks such as machine translation, summarization, question answering, and text generation.

Note: As of March 2022,